In [ ]:
from pathlib import Path
import os 

import matplotlib.pyplot as plt
import numpy as np
import auxiliary_functions as aux

import sirf.Gadgetron as pMR

# this is where we store the properly formatted data
root_path = Path(os.getenv("SIRF_INSTALL_PATH"))
root_path = root_path / "share/SIRF-3.1/Simulation/"
fpath_input = root_path / "Input"
fpath_output = root_path / "Output"

fname_stat          = fpath_output / "output_d_simulate_mrf_static_1500.h5"
fname_resp          = fpath_output / "output_g_simulate_mrf_resp.h5"
fname_cardioresp    = fpath_output / "output_g_simulate_mrf_cardio_resp.h5"

In [ ]:
ad_stat = pMR.AcquisitionData(str(fname_stat))
ad_resp = pMR.AcquisitionData(str(fname_resp))
ad_cardioresp = pMR.AcquisitionData(str(fname_cardioresp))

num_recon_imgs = 5
num_cg_iter = 10

recon_stat, ad_stat  = aux.iterative_reconstruct_timeresolved(ad_stat,num_recon_imgs,num_cg_iter)
recon_resp, ad_resp  = aux.iterative_reconstruct_timeresolved(ad_resp,num_recon_imgs,num_cg_iter)
recon_cardioresp, ad_cardioresp = aux.iterative_reconstruct_timeresolved(ad_cardioresp,num_recon_imgs,num_cg_iter)

In [ ]:
ad_cardioresp.number()

In [ ]:
# finally perform dictionary matching
fname_dict = Path("/media/sf_CCPPETMR/TestData/Input/xDynamicSimulation/pDynamicSimulation/Fingerprints/dict_70_1500.npz")
mrfdict = np.load(fname_dict)

dict_theta = mrfdict['dict_theta']
dict_mrf = mrfdict['dict_norm']

dict_stat = np.transpose( aux.apply_databased_sliding_window(ad_stat, np.transpose(dict_mrf)))
dict_resp = np.transpose( aux.apply_databased_sliding_window(ad_resp, np.transpose(dict_mrf)))
dict_cardioresp = np.transpose( aux.apply_databased_sliding_window(ad_cardioresp, np.transpose(dict_mrf)))

In [ ]:
match_stat = aux.dictionary_matching(recon_stat, dict_stat, dict_theta)
match_resp = aux.dictionary_matching(recon_resp, dict_resp, dict_theta)
match_cardioresp = aux.dictionary_matching(recon_cardioresp, dict_cardioresp, dict_theta)

In [ ]:
f,ax = plt.subplots(2,3)

ax[0,0].imshow(match_stat[:,:,1],cmap='jet',vmin=400,vmax=1500)
ax[1,0].imshow(match_stat[:,:,2],cmap='viridis',vmin=10,vmax=150)

ax[0,1].imshow(match_resp[:,:,1],cmap='jet',vmin=400,vmax=1500)
ax[1,1].imshow(match_resp[:,:,2],cmap='viridis',vmin=10,vmax=150)

ax[0,2].imshow(match_cardioresp[:,:,1],cmap='jet',vmin=400,vmax=1500)
ax[1,2].imshow(match_cardioresp[:,:,2],cmap='viridis',vmin=10,vmax=150)

plt.show()